In [ ]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

# Configure for presentation
np.set_printoptions(threshold=50, linewidth=50)
import matplotlib as mpl
mpl.rc('font', size=16)

## Census 2: Age and Gender in Census Data

In [ ]:
# this census file is online here: 
# http://www2.census.gov/programs-surveys/popest/datasets/2010-2015/national/asrh/nc-est2015-agesex-res.csv

# This file has same content as the one we explored in the Census 1 exercise
full_census_table = Table.read_table('nc-est2015-agesex-res.csv')
partial = full_census_table.select('SEX', 'AGE', 'POPESTIMATE2010', 'POPESTIMATE2015')
us_pop = partial.relabeled(2, '2010').relabeled(3, '2015')
us_pop

In [ ]:
us_pop.where('AGE', 70) # examine a set of rows with the same age

In [ ]:
us_pop.where('AGE', 70).where([False, True, True]) # remove the "all" zero element explicitly

In [ ]:
seventy = us_pop.where('AGE', 70)  # another comparison across the rows
seventy.column('2010') < 2000000

In [ ]:
seventy.where(seventy.column('2010') < 2000000)  # another way to remove a row

In [ ]:
us_pop.column('2015') / us_pop.column('2010') > 1.5  # compute a quantity and compare it to get a selector

In [ ]:
us_pop.where(us_pop.column('2015') / us_pop.column('2010') > 1.5)

## Census & ZIP Codes

In [ ]:
# Read a GIS file of all California Zip (postal) codes and their locations
zips = Map.read_geojson('ca_zips.geojson.gz')
zips

In [ ]:
zips_table = Table.from_records(zips.features)
zips_table.show(1)
# you may have to scale out and move to lower right to see the content

In [ ]:
# Berkeley's town postal codes are between 94700 and 94799
berkeley = zips_table.where('ZIP', are.between('94700', '94799'))
berkeley.show(2)
# you may have to scale out and move to lower right to see the content

In [ ]:
# You can ask for a map of all the entries in a table.
# The "feature" column is the GIS info
Map(berkeley.column('feature'))

### Income data

In [ ]:
# Read a separate table from the tax authority on total income per zip (in thousands)
income_raw = Table.read_table('ca_income_by_zip.csv', dtype={'ZIP': str})
income_raw

In [ ]:
# Reformat the data
# Drop some columns and sum the rows for each zip code
income_by_zipcode = income_raw.drop('STATEFIPS', 'STATE', 'agi_stub').group('ZIP', sum)
# Make a Table with the rows we'll need (column definitions from tax authority documentation)
income = Table().with_columns(
        'ZIP', income_by_zipcode.column('ZIP'),
        'returns', income_by_zipcode.column('N02650 sum'),
        'total income', income_by_zipcode.column('A02650 sum') * 1000, # data was in thousands
        'farmers', income_by_zipcode.column('SCHF sum')
    ).where('ZIP', are.not_equal_to('99999'))  # trim off out-of-state returns
income.set_format('total income', NumberFormatter(0)).show(5) # pretty print

In [ ]:
# What does the distibution of return (i.e. family) income look like:
plots.hist(income['total income'] / income['returns'], bins = 50);

In [ ]:
# Which zip codes have the highest total income?
income.sort('total income', descending=True)

In [ ]:
# Map that highest one manually
Map(zips_table.where('ZIP', '94301').column(1))
# It's Palo Alto, next to Stanford University and in Silicon Valley

In [ ]:
# Combine income and GIS data with a join operation, then find the highest (i.e. not manually)
income.join('ZIP', zips_table).sort('total income', descending=True)
# Again, move to the lower right to see the selected areas

In [ ]:
# Map all of the ZIP codes that have more than $1B returns.
Map(income.where('total income', are.above(1e9)).join('ZIP', zips_table).column('feature'))

In [ ]:
# Map all of the ZIP codes that have _average_ income over $1M
result = income.where( income['total income'] / income['returns'] > 1e6)
Map(result.join('ZIP', zips_table).column('feature'))
# Change that to $500k and repeat it.
# Try a few smaller ones too to see how the top end of income is distributed in California

In [ ]:
# Repeat that for average income under $30K
# (There aren't many tax returns under $20k due to the way US taxes work)
result = income.where( income['total income'] / income['returns'] < 30000)
Map(result.join('ZIP', zips_table).column('feature'))
# What do you notice about the kinds of places that show up?
# Try a few higher values to see how the low end of income is distributed in California

In [ ]:
# Some of the areas in the map above are city cores (except San Francisco and San Jose?), and 
# some appear to be farmland.  Let's look at where farmers are.

# Map all of the ZIP codes that have more than 5% farm income
f = income.where(income.column('farmers') >= 0.05 * income.column('returns'))
Map(f.join('ZIP', zips_table).column('feature'))

In [ ]:
# Map some with the highest number of farmers
f = income.where(income.column('farmers') >= 250)
Map(f.join('ZIP', zips_table).column('feature'))

In [ ]:
# Map all of the ZIP codes that have more than 5% farm income and $100M in returns
f = income.where(income.column('farmers') >= 0.05 * income.column('returns'))
both = f.where('total income', are.above(1e8))
Map(both.join('ZIP', zips_table).column('feature'))

In [ ]:
# For something completely different:
home = income.where('ZIP', '94707') # Bob's home zip code
home

In [ ]:
(home[2]/home[1])[0]   # average return (i.e. family) income (thosands)

In [ ]:
# Where does that lie in the distribution of family income in Calfornia?

In [ ]:
plots.hist(income['total income'] / income['returns'], bins = 50);
# As we saw, values go up over 1000 ($1B), but most are well below

In [ ]:
# find the fraction of California zip codes with smaller average income
below = income.where(income['total income'] / income['returns'] < (home[2]/home[1])[0])
below.num_rows / income.num_rows

In [ ]:
# Find the mean family income
# Hint:  Add family income to the table as a column so you can ask for mean()
income = income.with_column('family income', income['total income'] / income['returns'])
income['family income'].mean()